In [7]:
import boto3
import zipfile
import json
import io
import spacy
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
## s3 host to access data from UMIACS
s3_host = 'https://obj.umiacs.umd.edu'
access_key_id = "xxxxx"
secret_access_key = "xxxxx"

s3 = boto3.client('s3', 
                  endpoint_url=s3_host, 
                  aws_access_key_id=access_key_id, 
                  aws_secret_access_key=secret_access_key)


In [9]:
## relevant substrings related to BRI Projects from the entities we identified
## https://docs.google.com/document/d/1yoPZqhQqdBlaHAKHMbO3QpRmJVN1WLX8tmDwZ6sVDm0/edit

relevant_PHL_substrings = {"wawa", "clark", "subic", "powerplant", "pulangi", "negros", "snrdp", "national road", "development project", 
                           "rockwell", "estrella", "batangas", "phase 8", "ncr", "bss", "biopower", "biomass", "pnoc", "pv park", "meralco"
                            "marawi", "mapanuepe", "manila", "railway", "hydropower", "isabela", "power plant", "fdc", "diduyon", "global city",
                            "luzon", "Bukidnon", "bgc", "bonifacio global city", "origas", "binondo","banaoang", "angat", "amburayan", "agus", 
                            "agno river", "8a", "kauswagan", "kaliwa","general santos", "expressway", "chico river", "calaca", "bukidnon", 
                            "bonifacio"}
relevant_MLS_substrings = {"4b", "edra", "tnb", "kedah", "sungai", "siput", "south china sea", "malacca", "corridor","east coast", "railway link", 
                           "ecrl", "rembau", "penang", "sapangar", "samalaju", "pv park", "pantai", "murum", "malacca", "power plant", 
                           "industrial park", "biopower", "kuantan", "muda", "kota", "rapid transit", "klang", "wharf", "ppp", "kemaman port", 
                           "jimah", "gemas", "johor", "dungun", "bintulu", "baleh", "bakun", "machang"}
relevant_IDN_substrings = {"power plant", "sulawesi", "teluk sirih", "tayan", "riau", "awar","takalar", "suralaya", "madura", "sumatra", "sumsel",
                           "sulawesi", "sulut", "kertosono", "rembang", "pelabuhan", "ratu", "parit", "baru", "kalimantan", "pangkalan", "susu",
                           "paiton", "pacitan", "morowali", "industrial park", "aceh", "epc", "nagan", "raya", "power station", "manado", 
                           "road project", "lontar", "labuan", "angin", "kayan", "hydropower", "kalbar", "kalimantan", "jeneponto", "jatigede",
                           "jakarta", "speed railway", "ppp", "bandung", "toll road section", "gunturharjo", "indramayu", "cilacap", "bali", 
                           "kalimantan", "batang", "baten", "serang", "banjarsari", "balikpapan", "adipala"}
# relevant_substrings_combined = [relevant_PHL_substrings, relevant_MLS_substrings, relevant_IDN_substrings]

In [11]:
## helper method to look for substrings from a certain set in a given text
def look_for_substrings(relevant_substrings_set, tweet, project_country):
    ## data in the format of: tweet-id, includes-project-substring, tweet-text, tweet-language, project-country
    data = []
    for substring in relevant_substrings_set:
        raw_text = tweet["tweet_text"]
        if type(raw_text) != str:   ## one entry that has "nan" as the tweet text
            raw_text = str(raw_text)
        if substring in raw_text:
            row = (tweet["tweet_id"], substring, raw_text, tweet["tweet_language"], project_country)
            data.append(row)
    return data


In [12]:
## processes tmrc dataset
def process_tmrc(folder_prefix):
    ## we will collect the data with a list then create df at the end (most efficient with runtime)
    data = []
    response = s3.list_objects_v2(Bucket="twitter.tmrc", Prefix=folder_prefix)
    files = []
    ## collect all the filenames to be processed 
    for obj in response.get('Contents', []):
        object_key = obj['Key']
        if object_key.endswith('.zip'):
            files.append(object_key)
    ##  process each file
    for file in files:
        zip_object = s3.get_object(Bucket="twitter.tmrc", Key=file)
        zip_contents = zip_object['Body'].read()
        zip_file = zipfile.ZipFile(io.BytesIO(zip_contents), 'r')
        for file_info in zip_file.infolist():
            with zip_file.open(file_info) as json_file:
                file_name = file_info.filename                    
                ## we are only interested in the tweet file
                if not file_name.endswith("-tweet.json"):
                    continue
                try:
                    json_data = json_file.read().decode('utf-8')
                except: 
                    print("this is a text file")
                parsed_data = json.loads(json_data)
                for ind_data in parsed_data:
                    tweet = ind_data["tweet"]
                    data.extend(look_for_substrings(relevant_PHL_substrings, tweet, "PHL"))
                    data.extend(look_for_substrings(relevant_IDN_substrings, tweet, "IDN"))
                    data.extend(look_for_substrings(relevant_MLS_substrings, tweet, "MLS"))

                    # for camp_substr in relevant_substrings_combined:
                    #     data.extend(look_for_substrings(camp_substr, tweet, ))
                    # lang = ind_data['tweet']['tweet_language'] 

    return data

In [13]:
def getdf_twitterei(file):
    zip_object = s3.get_object(Bucket='twitter.ei', Key=file)
    zip_contents = zip_object['Body'].read()
    zip_file = zipfile.ZipFile(io.BytesIO(zip_contents), 'r')
    for file_info in zip_file.infolist():
        with zip_file.open(file_info) as csv_file:
            df = None
            try:
                if df == None:
                    df = pd.read_csv(csv_file)
                else:
                    df = pd.concat([df, pd.read_csv(csv_file)], axis=0)
            except:
                print(f"{csv_file.filename} is not a csv file")
    df["tweet_id"] = df["tweetid"]
    return df[["tweet_id", "tweet_language", "tweet_text"]]

def process_twitterei(df):
    acc = []
    df.apply(lambda row: process_row(row, acc), axis=1)
    return acc

def process_row(row, acc):
    acc.extend(look_for_substrings(relevant_PHL_substrings, row, "PHL"))
    acc.extend(look_for_substrings(relevant_IDN_substrings, row, "IDN"))
    acc.extend(look_for_substrings(relevant_MLS_substrings, row, "MLS"))



In [14]:
tmrc_folder_prefix_lst = ['August_2022/TMRC14_APAC_1/', 
                          'August_2022/TMRC14_APAC_2/', 
                          'October_2022/TMRC15_APAC_3/']
twitterei_folder_prefix_lst = ["2019_08/china_082019_1/china_082019_1_tweets_csv_unhashed.zip",
                               "2019_08/china_082019_2/china_082019_2_tweets_csv_unhashed.zip",
                               "2019_08/china_082019_3/china_082019_3_tweets_csv_unhashed.zip",
                               "2020_05/china_052020/china_052020_tweets_csv_unhashed.zip", 
                               "2020_09/thailand_092020/thailand_092020_tweets_csv_unhashed.zip"]

## this dict will be in the format of campaign_name, data
campaign_data_dict = dict()


In [15]:
for f in tmrc_folder_prefix_lst:
    name = f.split('/')[1]
    print(name)
    campaign_data_dict[name] = process_tmrc(f)


for f in twitterei_folder_prefix_lst:
    name = f.split('/')[1]
    print(name)
    df = getdf_twitterei(f)
    campaign_data_dict[name] = process_twitterei(df)


TMRC14_APAC_1
TMRC14_APAC_2
TMRC15_APAC_3
china_082019_1


/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_39325/1549733038.py:10: DtypeWarning: Columns (6,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


china_082019_2


/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_39325/1549733038.py:10: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


china_082019_3


/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_39325/1549733038.py:10: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
/var/folders/r4/sgr1zrxx0038xwsxj6n3k8900000gn/T/ipykernel_39325/1549733038.py:10: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


china_052020
thailand_092020


In [16]:
## turn dict into dataframe
text_relevant_df = pd.DataFrame()
for campaign, list in campaign_data_dict.items():
    ## data in the format of: CampaignID, tweet-id, includes-project-substring, tweet-text, tweet-language, project-country
        cur_df = pd.DataFrame(list)
        cur_df.columns = ['tweet_id', 'project_substring', 'tweet_text', 'tweet_language', 'project_country']
        cur_df["CampaignID"] = campaign
        text_relevant_df = pd.concat([text_relevant_df, cur_df], ignore_index=True)
text_relevant_df

,tweet_id,project_substring,tweet_text,tweet_language,project_country,CampaignID
0,1387933360429748225,muda,Munarman telah diciduk ini lah dukungan dri an...,in,MLS,TMRC14_APAC_1
1,1366595382596149248,baru,RT @radenrauf: Apa yang bikin kamu gampang ilf...,in,IDN,TMRC14_APAC_1
2,1379423796357001218,angat,tegakan #HukumHRS jangan biarkan bebas provoka...,in,PHL,TMRC14_APAC_1
3,1382310663410774021,baru,"Rendahkan Martabat peradilan, Rizieq Shihab di...",in,IDN,TMRC14_APAC_1
4,1383295471058259970,baru,Terbukti nyata bhw pemrintah terbuka terhdp ma...,in,IDN,TMRC14_APAC_1
...,...,...,...,...,...,...
198836,1215191677482127360,8a,#เมียหลวงบวกเมียน้อย https://t.co/IWZW7tp8aa,und,PHL,thailand_092020
198837,1227491832889634817,8a,#ผบทบต้องลาออก แล้วเสรีรวมไอซ์ทำไมไม่ออก 555 h...,th,PHL,thailand_092020
198838,1192277843427749888,raya,น้ำตาคนลำพระยา จ.ยะลา\n#Savelampraya #Saveyala...,th,IDN,thailand_092020
198839,1197394755228798976,8a,https://t.co/lomfgaqMyW https://t.co/8aAzjAXcJl,und,PHL,thailand_092020


In [17]:
# text_relevant_df.to_csv("~/Coding/buntain/sentiment_data/text_relevant_df.csv", index=False)
text_relevant_df.to_csv("~/Coding/buntain/sentiment_data/text_relevant_df.csv", encoding='utf-8', quoting=csv.QUOTE_ALL)


In [18]:
text_relevant_df["project_substring"].value_counts().head(15)

project_substring
bali      47871
baru      47803
ncr       10486
awar       9707
4b         8579
angat      8445
agus       8040
8a         7754
muda       7610
angin      7410
ratu       6182
kota       4462
penang     3410
ppp        3040
raya       2165
Name: count, dtype: int64

In [19]:
text_relevant_df["project_country"].value_counts()

project_country
IDN    132084
PHL     38378
MLS     28379
Name: count, dtype: int64

In [20]:
text_relevant_df["tweet_language"].value_counts()

tweet_language
in     144025
en      22418
zh       4597
und      3290
ar       2754
pt       2574
tl       1972
es       1201
id        483
ko        291
fr        262
ja        220
hi        206
ht        188
et        173
nl        147
tr        111
th         92
fa         91
ro         86
it         81
pl         68
no         66
fi         64
de         62
cy         51
eu         46
da         37
ur         36
lt         33
cs         33
lv         30
ru         28
hu         27
sv         24
zxx        19
ca         17
sl         13
is         12
qme        11
vi          9
art         4
hr          4
bn          4
sk          3
qht         2
qam         1
mr          1
bs          1
ne          1
bg          1
Name: count, dtype: int64

In [21]:
text_relevant_df["CampaignID"].value_counts()

CampaignID
china_082019_1     100410
china_082019_3      60203
china_082019_2      16910
TMRC14_APAC_1        8435
TMRC14_APAC_2        7915
TMRC15_APAC_3        3271
china_052020         1630
thailand_092020        67
Name: count, dtype: int64